In [1]:
import h5py,numpy as np,os
path = '../data/fold0_noFIR.mat'
data = h5py.File(path, 'r')

x_train = data['trainX'][:].astype('float32')
x_train = np.expand_dims(x_train.transpose(),1)

x_val = data['valX'][:].astype('float32')
x_val = np.expand_dims(x_val.transpose(),1)

y_train = data['trainY'][:].astype('int32')
y_train = y_train.transpose()
y_train = y_train[:,0]
y_train[y_train<0] = 0

y_val = data['valY'][:].astype('int32')
y_val = y_val.transpose()
y_val = y_val[:,0]
y_val[y_val<0] = 0

val_parts = data['val_parts'][:].astype('int32').squeeze(0)
domain = np.array([x-97 for x in data['val_files'][:][0]])
train_domain = np.array([x-97 for x in data['train_files'][:][0]])

In [3]:
import torch.optim as optim
import torch
from torch.autograd import Variable
import time
import torch
import torch.nn as nn
# from torchvision import datasets, transforms
from torchsummary import summary
import numpy as np
from HeartCepTorch import MFCC_Gen,Network
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn.init as init
from torchsummary import summary

# from utils import log_macc, results_log
from dataLoader import reshape_folds
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import matplotlib.pyplot as plt
# import seaborn as sns
# import Evaluator
import dataLoader
# from custom_layers import Attention
from sklearn.model_selection import train_test_split
import seaborn as sns
from collections import Counter
def to_numpy(x):
    return x.cpu().detach().numpy()
def plotf(x):
    plt.plot(to_numpy(x))

In [4]:
model = Network(2,0)
mfcc_gen = MFCC_Gen(fs=1000,filters=64,momentum=0.1)
model.cuda()
mfcc_gen.cuda()
class_criterion = nn.CrossEntropyLoss()
domain_criterion = nn.CrossEntropyLoss()
# model.load_state_dict(torch.load('../models/weights.0580-acc_0.7824-macc_0.8294.pt'))
model.load_state_dict(torch.load('../models/weights.0150-acc_0.7990-macc_0.8562.pt'))


<All keys matched successfully>

In [5]:
from torch.utils.data import Dataset, DataLoader

In [6]:
class HS_dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.x[idx],self.y[idx]
dataset = HS_dataset(x_train,y_train)

In [7]:
loader = DataLoader(dataset, batch_size=500, shuffle=False, sampler=None,
           batch_sampler=None, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)

In [8]:
epoch_loss = 0
acc = 0
N = 0
y_true = None
feature = None
model.eval()
mfcc_gen.eval()
with torch.no_grad():
    for (x,y) in iter(loader):
#         x,y = torch.from_numpy(x),torch.from_numpy(y)
        x = x.type(torch.FloatTensor).cuda()
        x = mfcc_gen(x)
        x = x.transpose(2,1)
        x = x.unsqueeze(1)
        x,y = Variable(x),Variable(y)
        y = y.long().cuda()
        cls= model(x).float()
        feat = model.extractor(x)
        
        if(feature is None):
            feature = to_numpy(feat)
        else:
            feature = np.concatenate(feature,to_numpy(feat))
        if(y_pred is None):
            y_true = to_numpy(y)
        else:
            y_true = np.concatenate(y_true,to_numpy(y))
        

RuntimeError: CUDA out of memory. Tried to allocate 932.00 MiB (GPU 0; 3.00 GiB total capacity; 1.42 GiB already allocated; 372.36 MiB free; 1.70 GiB reserved in total by PyTorch)